<br>
<br>
# __'Fresh & Fitness Co'__
<br>
<br>

### Business Introduction  


__<font color=green>'Fresh & Fitness Co'</font>__  is a private UK based company specialized in producing 100% fresh fruit/vegetable juices and beverages and selling them primarily in thier fitness facilities. It has presence in over 25 locations in London, Liverpool and Manchester in UK with annual revenue of £40 million and over 400 employees. 

### Market segmentation

It targets mostly top 25% of the population with an annual income of £75,000 and ages between 25-55. __<font color=green>'Fresh & Fitness Co'</font>__  has been quite successfull in terms of retaining higher turnout rate of its members (93.5%) compared to other private fitness halls (85%) in the market.

### Products and Facilities

__<font color=green>'Fresh & Fitness Co'</font>__  only uses organic fruits and vegetables produced by local farmers only for its juices and protein beverages. In addition to regular gym facilities and swimming pool, it offers wide range of services such as private cross-fit and yoga classes, oriental meditation and cardio dances for all its premium members.

### Business Problem (International Expansion)

__<font color=green>'Fresh & Fitness Co'</font>__ has been planning to expand its business internationally for few years already. Its first primary target been the state of New York, US where it is planning to open a test facility. Despite the fierce competetion in US fitness market, the company is quite confident in its success and ability to differentiate itself from other market participants.

#### However, the company has following questions/problems to analyze before it decides to enter US market:
	
_<font color=blue>1. Which area/neighborhood to enter in New York, US?_</font>  
_<font color=blue>2. What is the most popular venue in the neighborhood in the area chosen?_</font>    
_<font color=blue>3. What is the demographics/overall health condition of the area?_</font>  
_<font color=blue>4. How healthy is the diet of the population?_</font>    
_<font color=blue>5. How likely is it have a high member turnover rate?_</font>    

## For the Business analysis I will be using the following data below:

 #### 1. NY State Demographic data 
 #### 2. NY State: Age adjusted percentage of adults obese (BMI 30 or higher)
 #### 3. NY State: Age adjusted Percentage of adults eating 5 or more fruits or vegetables per day
 #### 4. NY State: Age adjusted Percentage of adults who did not participate in leisure time physical activity in last 30 days
 #### 5. Location data derived from the main data (for Foursquare API)
 <br>
 <br>

In [249]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import urllib, json


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import geocoder
import csv

### NY State Demographic data 


In [3]:
ny_demographic_data = pd.read_html('https://www.health.ny.gov/statistics/vital_statistics/2009/table02.htm')
ny_demographic_data = pd.DataFrame(ny_demographic_data[0])
ny_demographic_data.drop(ny_demographic_data.index[0:4], inplace=True)
ny_demographic_data.drop(ny_demographic_data.index[5:7], inplace=True)
ny_demographic_data.reset_index(inplace=True, drop=True)
ny_demographic_data.drop('2000 Census Population', axis=1, inplace=True)
ny_demographic_data.rename(columns={"2009 Population Estimate": "Population", 
                                    "2000 Land Area Square Miles": "Area", 
                                    "2009 Population Density1": "Density"},
                                    inplace=True)
ny_demographic_data.head(10)

                    

,County,Population,Area,Density
0,Bronx,1397287,42.03,33247.32
1,Kings,2567098,70.61,36358.05
2,New York,1629054,22.96,70940.25
3,Queens,2306712,109.24,21116.93
4,Richmond,491730,58.48,8408.71
5,Albany,298284,523.45,569.85
6,Allegany,49157,1030.22,47.72
7,Broome,194630,706.82,275.36
8,Cattaraugus,79689,1309.85,60.84
9,Cayuga,79526,693.18,114.73


### NY State: Age adjusted percentage of adults obese (BMI 30 or higher)

In [4]:
ny_bmi_data = pd.read_csv(r'Community_Health___Age-adjusted_percentage_of_adults_obese__BMI_30_or_higher___2008_-_2009.csv')
ny_bmi_data = pd.DataFrame(ny_bmi_data)
ny_bmi_data.drop(['Indicator Number','Indicator','Quartile','Total Event Counts', 'Denominator', 'Denominator Note', 
                'Measure Unit', '95% CI', 'Data Comments', 'Data Years','Data Sources', 'Mapping Distribution'], axis=1, inplace=True)

ny_bmi_data.rename(columns={"County Name": "County", 
                                    "Region Name": "Region", 
                                    "Percentage/Rate": "BMI Rate"},
                                    inplace=True)

ny_bmi_data.drop(ny_bmi_data.index[-1], inplace=True)
ny_bmi_data.head(10)

,County,County Code,Region,BMI Rate,Location
0,Allegany,2,Western New York,30.5,"(42.226801, -78.020567)"
1,Cattaraugus,4,Western New York,31.7,"(42.224267, -78.606673)"
2,Chautauqua,6,Western New York,27.4,"(42.246904, -79.315313)"
3,Erie,14,Western New York,28.0,"(42.888143, -78.795544)"
4,Genesee,18,Western New York,27.6,"(43.000121, -78.172679)"
5,Niagara,29,Western New York,26.6,"(43.134634, -78.856419)"
6,Orleans,34,Western New York,29.1,"(43.248394, -78.218438)"
7,Wyoming,56,Western New York,31.6,"(42.723641, -78.204473)"
8,Chemung,7,Finger Lakes,30.0,"(42.116644, -76.812331)"
9,Livingston,24,Finger Lakes,22.7,"(42.763754, -77.765392)"


### NY State: Age adjusted Percentage of adults eating 5 or more fruits or vegetables per day

In [8]:
ny_fruits_data = pd.read_csv('Community_Health__Age-adjusted_Percentage_of_adults_eating_5_or_more_fruits_or_vegetables_per_day__2008_-_2009.csv')
ny_fruits_data = pd.DataFrame(ny_fruits_data)
ny_fruits_data.drop(['Indicator Number','Indicator','Quartile','Total Event Counts', 'Denominator', 'Denominator Note', 
               'Measure Unit', '95% CI', 'Data Comments', 'Data Years','Data Sources', 'Mapping Distribution'], axis=1, inplace=True)

ny_fruits_data.rename(columns={"County Name": "County", 
                                    "Region Name": "Region", 
                                    "Percentage/Rate": "Fruits & Vegetable Eating Rate"},
                                    inplace=True)
#ny_fruits_data['Indicator'][0]

ny_fruits_data.drop(ny_fruits_data.index[0], inplace=True)
ny_fruits_data.head()

,County,County Code,Region,Fruits & Vegetable Eating Rate,Location
1,Allegany,2,Western New York,21.2,"(42.226801, -78.020567)"
2,Cattaraugus,4,Western New York,20.0,"(42.224267, -78.606673)"
3,Chautauqua,6,Western New York,24.9,"(42.246904, -79.315313)"
4,Erie,14,Western New York,26.3,"(42.888143, -78.795544)"
5,Genesee,18,Western New York,27.1,"(43.000121, -78.172679)"


### NY State: Age adjusted Percentage of adults who did not participate in leisure time physical activity in last 30 days

In [44]:
ny_physactivity_data = pd.read_csv('Community_Health__Age-adjusted_Percentage_of_adults_who_did_not_participate_in_leisure_time_physical_activity_in_last_30_days__2008_-_2009.csv')
ny_physactivity_data = pd.DataFrame(ny_physactivity_data)
ny_physactivity_data.drop(['Indicator Number','Indicator','Quartile','Total Event Counts', 'Denominator', 'Denominator Note', 
               'Measure Unit', '95% CI', 'Data Comments', 'Data Years','Data Sources', 'Mapping Distribution'], axis=1, inplace=True)


ny_physactivity_data.rename(columns={"County Name": "County", 
                                    "Region Name": "Region", 
                                    "Percentage/Rate": "Physical Activity Rate"},
                                    inplace=True)
ny_physactivity_data.drop(ny_physactivity_data.index[-1], inplace=True)

ny_physactivity_data.head()


,County,County Code,Region,Physical Activity Rate,Location
0,Allegany,2,Western New York,75.0,"(42.226801, -78.020567)"
1,Cattaraugus,4,Western New York,78.4,"(42.224267, -78.606673)"
2,Chautauqua,6,Western New York,77.9,"(42.246904, -79.315313)"
3,Erie,14,Western New York,80.8,"(42.888143, -78.795544)"
4,Genesee,18,Western New York,74.0,"(43.000121, -78.172679)"


#### Merging NY State data into one table

In [45]:
ny_data_joined = pd.merge(ny_demographic_data, ny_bmi_data, on="County", how='inner')
ny_data_joined = pd.merge(ny_data_joined, ny_fruits_data, on="County", how='inner')
ny_data_joined = pd.merge(ny_data_joined, ny_physactivity_data, on="County", how='inner')
ny_data_joined.drop(['Region_x', 'Location_x','County Code_x', 'Region_y', 'Location_y', 'County Code_y'], axis=1, inplace=True)

ny_data_joined.head()

,County,Population,Area,Density,BMI Rate,Fruits & Vegetable Eating Rate,County Code,Region,Physical Activity Rate,Location
0,Bronx,1397287,42.03,33247.32,29.1,6.3,58,New York City,69.2,"(40.85589, -73.868294)"
1,Kings,2567098,70.61,36358.05,25.0,10.4,59,New York City,70.4,"(40.65642, -73.950691)"
2,New York,1629054,22.96,70940.25,16.1,13.6,60,New York City,80.2,"(40.726966, -74.005966)"
3,Queens,2306712,109.24,21116.93,20.6,6.8,61,New York City,71.9,"(40.749338, -73.789673)"
4,Richmond,491730,58.48,8408.71,26.6,10.0,62,New York City,70.2,"(40.566763, -74.148102)"


In [99]:
column_names = ny_data_joined.columns.to_list()
column_names = ['County', 'County Code','Region','Area','Density','Population','BMI Rate',
 'Fruits & Vegetable Eating Rate','Physical Activity Rate','Location']
ny_data_joined=ny_data_joined[column_names]
ny_data_joined.head(20)

,County,County Code,Region,Area,Density,Population,BMI Rate,Fruits & Vegetable Eating Rate,Physical Activity Rate,Location
0,Bronx,58,New York City,42.03,33247.32,1397287,29.1,6.3,69.2,"(40.85589, -73.868294)"
1,Kings,59,New York City,70.61,36358.05,2567098,25.0,10.4,70.4,"(40.65642, -73.950691)"
2,New York,60,New York City,22.96,70940.25,1629054,16.1,13.6,80.2,"(40.726966, -74.005966)"
3,Queens,61,New York City,109.24,21116.93,2306712,20.6,6.8,71.9,"(40.749338, -73.789673)"
4,Richmond,62,New York City,58.48,8408.71,491730,26.6,10.0,70.2,"(40.566763, -74.148102)"
5,Albany,1,Northeastern New York,523.45,569.85,298284,21.7,23.9,80.7,"(42.678066, -73.814233)"
6,Allegany,2,Western New York,1030.22,47.72,49157,30.5,21.2,75.0,"(42.226801, -78.020567)"
7,Broome,3,New York-Penn,706.82,275.36,194630,24.9,27.4,82.7,"(42.122015, -75.933191)"
8,Cattaraugus,4,Western New York,1309.85,60.84,79689,31.7,20.0,78.4,"(42.224267, -78.606673)"
9,Cayuga,5,Central New York,693.18,114.73,79526,27.6,27.0,77.1,"(42.940095, -76.560755)"


In [238]:
location_str = ''

for a in ny_data_joined['Location']:
     for b in a:
        if b not in ['(']:
             location_str += b
location_str = location_str.replace(')', ' ')
location_str = location_str.replace(',', '')

location = location_str.split(' ')
latitude = [location[a] for a in range(0,len(location), 2)]
latitude =latitude[:-1]
longitude = [location[a] for a in range(1,len(location), 2)]


ny_data_joined['Latitude'] = latitude
ny_data_joined['Longitude'] = longitude
ny_data_joined.drop('Location', axis=1, inplace=True)

In [245]:
ny_data_joined.head(10)

,County,County Code,Region,Area,Density,Population,BMI Rate,Fruits & Vegetable Eating Rate,Physical Activity Rate,Latitude,Longitude
0,Bronx,58,New York City,42.03,33247.32,1397287,29.1,6.3,69.2,40.85589,-73.868294
1,Kings,59,New York City,70.61,36358.05,2567098,25.0,10.4,70.4,40.65642,-73.950691
2,New York,60,New York City,22.96,70940.25,1629054,16.1,13.6,80.2,40.726966,-74.005966
3,Queens,61,New York City,109.24,21116.93,2306712,20.6,6.8,71.9,40.749338,-73.789673
4,Richmond,62,New York City,58.48,8408.71,491730,26.6,10.0,70.2,40.566763,-74.148102
5,Albany,1,Northeastern New York,523.45,569.85,298284,21.7,23.9,80.7,42.678066,-73.814233
6,Allegany,2,Western New York,1030.22,47.72,49157,30.5,21.2,75.0,42.226801,-78.020567
7,Broome,3,New York-Penn,706.82,275.36,194630,24.9,27.4,82.7,42.122015,-75.933191
8,Cattaraugus,4,Western New York,1309.85,60.84,79689,31.7,20.0,78.4,42.224267,-78.606673
9,Cayuga,5,Central New York,693.18,114.73,79526,27.6,27.0,77.1,42.940095,-76.560755


### Based on my main data, I derived the below location data to use Foursquare API to get the most popular venues in each county

In [247]:
county_location_data = ny_data_joined[['County', 'Region', 'Latitude', 'Longitude']]
county_location_data.head()

,County,Region,Latitude,Longitude
0,Bronx,New York City,40.85589,-73.868294
1,Kings,New York City,40.65642,-73.950691
2,New York,New York City,40.726966,-74.005966
3,Queens,New York City,40.749338,-73.789673
4,Richmond,New York City,40.566763,-74.148102
